In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import pymp 
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
from collections import defaultdict
from multiprocessing import Pool, Manager, cpu_count
from IPython.display import display_html
from itertools import chain, cycle
import lightgbm as lgb
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import *
from src.util import *
from src import config
from src.visualize import *
from src.decode import *
pd.set_option('display.float_format', lambda x: '%.04f' % x)
gezi.set_pandas_widder()

No module named 'tensorflow.python.keras.layers.preprocessing'


In [2]:
df_gt = pd.read_feather('../input/feedback-prize-2021/train_en.fea')

In [3]:
df_flat = pd.read_feather('../input/feedback-prize-2021/train_flat_en.fea')

In [4]:
df_flat.head(1)

,index,id,text,kfold,cluster,worker,part,para_type,start,end
0,0,0000D23A521A,"Some people belive that the so called ""face"" on mars was created by life on mars. This is not the case. The face on Mars is a naturally occuring land form called a mesa. It was not created by aliens, and there is no consiracy to hide alien lifeforms on mars. There is no evidence that NASA has found that even suggests that this face was created by aliens. [BR] [BR] A mesa is a naturally occuring rock formation, that is found on Mars and Earth. This ""face"" on mars only looks like a face because humans tend to see faces wherever we look, humans are obviously extremely social, which is why our brain is designed to recognize faces. [BR] [BR] Many conspiracy theorists believe that NASA is hiding life on Mars from the rest of the world. These people would be very wrong. If NASA found life on Mars, then they would get millions of people's attention. NASA's budget would increase drasticly, which means that their workers would get paid more. There is no good reason that NASA would hide life on Mars from the rest of the world. [BR] [BR] So, NASA is not hiding life on Mars from us, and they are not trying to trick us into thinking that the ""face"" on mars is just a mesa, because it actually is. NASA hiding life would be illogical, because if they found life on Mars, they would make a lot of money, and we all know that the people at NASA aren't illogical people.",0,7,10,2,"[3, 2, 2, 1, 6, 7, 2, 4]","[0, 34, 69, 84, 117, 134, 154, 186]","[34, 69, 84, 117, 134, 154, 186, 251]"


In [5]:
df_flat['num_words'] = df_flat['text'].apply(lambda x: len(x.replace('[BR]', '\n').split()))

In [6]:
df_gt.head(1)

,index,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker
0,0,423A1CA112E2,13,0,0,0,1,0,3.0000,Phones [BR] [BR],"Phones [BR] [BR] Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. [BR] [BR] When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation. [BR] [BR] Driving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems. That's why there's a thing that's called no texting while driving. That's a really important thing to remember. Some people still do it because they think It's stupid. No matter what they do they still have to obey it because that's the only way how did he save. [BR] [BR] Sometimes on the news there is either an accident or a suicide. It might involve someone not looking where they're going or tweet that someone sent. It either injury or death. If a mysterious number says I'm going to kill you and they know where you live but you don't know the person's contact [BR] [BR] ,It makes you puzzled and make you start to freak out. Which can end up really badly. [BR] [BR] Phones are fine to use and it's also the best way to come over help. If you go through a problem and you can't find help you ,always have a phone there with you. Even though phones are used almost every day as long as you're safe it would come into use if you get into trouble. Make sure you do not be like this phone while you're in the middle of driving. The news always updated when people do something stupid around that involves their phones. The safest way is the best way to stay safe.",0,2,8,67


In [7]:
v = 47
votes = gezi.read_pickle(f'../working/offline/{v}/votes.pkl')

In [8]:
dfs = Manager().dict()
with pymp.Parallel(len(votes)) as p:
  for i in p.range(len(votes)):
    vote = votes[i]
    fes = []
    for id in tqdm(vote, total=len(vote)):
      fe = {}
      fe['id'] = id
      fe['model'] = i
      gt = get_gt_dict(df_gt, id)
      num_gts = 0
      for key in gt:
        num_gts += len(gt[key])
      fe['num_gts'] = num_gts
      res_list = vote[id]
      pred = list_to_dict(res_list)
      fe['score'] = essay_f1(gt, pred, return_dict=False)
      fe['cls'] = []
      fe['len'] = []
      fe['sep_prob'] = []
      fe['token_logits'] = []
      fe['token_probs'] = []
      for res in res_list:
        fe['cls'].append(res['cls'])
        fe['len'].append(res['len'])
        fe['sep_prob'].append(res['sep_prob'])
        fe['token_logits'].append(res['token_logits'])
        fe['token_probs'].append(res['token_prob'])
      fe['num_paras'] = len(fe['cls'])
      fes.append(fe)
    d = pd.DataFrame(fes)
    dfs[i] = d

  0%|          | 0/15594 [00:00<?, ?it/s]

In [9]:
dfs[0].score.mean()

0.710594286387137

In [10]:
dfs[1].score.mean()

0.6319851517440416

In [11]:
dfs[13].score.mean()

0.6975040916905079

In [12]:
df = pd.concat([df for x, df in dfs.items()])

In [13]:
df.head(1)

,id,model,num_gts,score,cls,len,sep_prob,token_logits,token_probs,num_paras
0,0000D23A521A,5,8,0.5238,"[5, 3, 1, 2, 1, 2, 6, 7, 2, 4]","[21, 13, 17, 18, 15, 33, 17, 20, 32, 65]","[0.9144795269866627, 0.8629530995247405, 0.7717764474869726, 0.9963936547909024, 0.8864285333444537, 0.9983718313682012, 0.9548764752687496, 0.1772727876030648, 0.9980646076789675, 1.0]","[[0.3779178722983315, -1.6021377472650438, -2.4511854364758445, 2.75136205128261, -3.1190939176650274, 6.784086636134556, -0.9309209727105641, -2.6760353360857283], [-1.6985660974796002, 1.3281200298896203, 0.03521042001935152, 6.736131227933443, -3.4585539377652683, 1.5385815455363347, -3.3237388684199405, -1.8013806343078613], [-0.16776051306549242, 3.889273489222807, 3.2884954003726734, 2.2212097434436573, -4.279685974121094, -0.24686236854861765, -3.0662520633024326, -2.8563820053549374], [1.5465700692600675, 3.2572935157352023, 3.8807343509462147, 1.8826661441061232, -3.5286709202660456, -2.633055461777581, -2.761381361219618, -3.0761695437961154], [1.3244357268015543, 6.355816173553467, 3.734326887130737, -1.7101484616597493, -2.9606508096059163, -3.9881274700164795, -1.8525549650192261, -2.0613436778386434], [1.1095339872620322, 3.738290700045499, 7.229684627417362, -2.82918381690979, -3.2120424834164707, -3.498941081942934, -1.910789020133741, -2.065453189792055], [-0.5392662996754927, 0.7079517175169552, 1.1538029488395243, -1.8988807411754833, -1.873082020703484, -2.3295224273906037, 7.562848175273222, -1.8861653594409717], [-1.647926214337349, 0.8645792812108993, 3.0301221292465925, -3.07043097615242, -2.8233571290969848, -3.802316415309906, -0.9449738070368767, 5.793924880027771], [-0.45701957686105743, 2.372330540791154, 5.448880508542061, -3.096696086227894, -3.3657204806804657, -4.503334306180477, -2.5151140093803406, 3.801049843430519], [-0.13664574153148212, -0.4811398635403468, -0.12540175573757062, -0.8136148671118113, 7.412513322096604, -4.16263709801894, -2.213939362305861, -1.7650066669170672]]","[[0.03597789997793094, 0.005123395518389092, 0.00224702301398622, 0.42297086540757856, 0.001066609953361114, 20.520644611116158, 0.009420556089135567, 0.0025490389234554627], [0.0032720686513563408, 0.05854620507546927, 0.015958014396855562, 12.846224753073002, 0.0005449907851375334, 0.07222312862718198, 0.0005797487451693214, 0.0026510906458286406], [0.28519296332364474, 9.504166573912471, 5.213210986863298, 1.8180320737127278, 0.002952158247614629, 0.15523266783210912, 0.009459256004472796, 0.01175332010366096], [0.989376424583566, 5.4623483627987115, 10.130964384154739, 1.3726218657404994, 0.006291668732230256, 0.015270996346196846, 0.013389219467480253, 0.009737078176575348], [0.09176379585072794, 13.878040284982458, 1.016766627728418, 0.004418548335143659, 0.0013258267631666202, 0.0004645488979186826, 0.00406948016590671, 0.003150887276260236], [0.0717915748991207, 0.9995492504482303, 31.918981517461567, 0.0014458678604222497, 0.0010122520787608272, 0.0007135357341694472, 0.003514443001647113, 0.002991558516089058], [0.005842686912692666, 0.018168354566757278, 0.02818684870528511, 0.001419479473214986, 0.0016176657160622018, 0.0009114656923993313, 16.9424111386575, 0.0014423602760861365], [0.013646131561206632, 0.15470972451665302, 2.7497039289186342, 0.0034723997795879378, 0.004219115012542414, 0.001559211399901754, 0.042943164047093765, 17.029746324764375], [0.10474905701341358, 2.0845112330929507, 23.703433758579003, 0.00632823985589014, 0.003615804257959434, 0.00119435774396254, 0.008660986905472817, 6.0875065625513445], [0.04347803737602476, 0.025488053400450152, 0.036846728252037234, 0.02030724286573219, 64.86146545968259, 0.0006594257016813699, 0.004743752472459133, 0.0070113002490211065]]",10


In [14]:
df = df.merge(df_flat[['id', 'kfold']], on='id')

In [15]:
df[(df.model==0) & (df.kfold==0)]['score'].mean()

0.7122536918869941

In [16]:
df = df.merge(df_flat[['id', 'num_words']], on='id')

In [60]:
def gen_feat(row):
  fe = {}
  fe['id'] = row['id']
  fe['fold'] = row['kfold']
  fe['score'] = row['score']
  fe['model'] = row['model']
  fe['num_paras'] = row['num_paras']
  fe['num_words'] = row['num_words']
  fe['num_gts'] = row['num_gts']
  try:
    sep_probs = np.array(row['sep_prob'][:-1])
    fe['sep_prob_min'] = sep_probs.min()
    fe['sep_prob_max'] = sep_probs.max()
    fe['sep_prob_mean'] = sep_probs.mean()
    classes = row['cls']
    lens = np.array(row['len'])
    fe['len_min'] = lens.min()
    fe['len_max'] = lens.max()
    fe['len_mean'] = lens.mean()
    token_probs = gezi.softmax(row['token_probs'])
    token_probs = np.array([prob[cls_] for cls_, prob in zip(classes, token_probs)])
    fe['token_probs_min'] = token_probs.min()
    fe['token_probs_max'] = token_probs.max()
    fe['token_probs_mean'] = token_probs.mean()
    
    bigram_probs = []
    for i in range(1, len(classes)):
      a, b = classes[i - 1], classes[i]
      a, b = id2dis[a], id2dis[b]
      pair = f'{a}|{b}'
      bigram_probs.append(bigrams.get(pair, 0) / unigrams[a])
    bigram_probs = np.array(bigram_probs)
    fe['bi_probs_min'] = bigram_probs.min()
    fe['bi_probs_max'] = bigram_probs.max()
    fe['bi_probs_mean'] = bigram_probs.mean()
      
  except Exception:
    fe.update({
      'sep_prob_min': 0,
      'sep_prob_max': 0,
      'sep_prob_mean': 0,
      'len_min': 0,
      'len_max': 0,
      'len_mean': 0,     
      'token_probs_min': 0,
      'token_probs_max': 0,
      'token_probs_mean': 0,
      'bi_probs_min': 0,
      'bi_probs_max': 0,
      'bi_probs_mean': 0,
    })
  return fe

In [61]:
fes = []
for row in tqdm(df.itertuples(), total=len(df)):
  row = row._asdict()
  fes.append(gen_feat(row))

  0%|          | 0/265098 [00:00<?, ?it/s]

In [62]:
dall = pd.DataFrame(fes)

In [63]:
dall

,id,fold,score,model,num_paras,num_words,num_gts,sep_prob_min,sep_prob_max,sep_prob_mean,len_min,len_max,len_mean,token_probs_min,token_probs_max,token_probs_mean,bi_probs_min,bi_probs_max,bi_probs_mean
0,0000D23A521A,0,0.5238,5,10,251,8,0.1773,0.9984,0.8401,13,65,25.1000,0.9856,1.0000,0.9976,0.0736,0.6168,0.3648
1,0000D23A521A,0,0.4857,2,10,251,8,0.3232,0.9960,0.7723,13,65,25.1000,0.0002,1.0000,0.8999,0.0736,0.6168,0.3268
2,0000D23A521A,0,0.3810,4,9,251,8,0.4948,0.9975,0.8578,13,65,27.8889,0.7886,1.0000,0.9757,0.0736,0.5574,0.3093
3,0000D23A521A,0,0.4857,14,10,251,8,0.2536,0.9987,0.7987,13,65,25.1000,0.0000,1.0000,0.8683,0.0736,0.6168,0.3268
4,0000D23A521A,0,0.6286,15,9,251,8,0.0002,0.9951,0.7014,15,65,27.8889,0.0001,1.0000,0.8889,0.0736,0.6168,0.3293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265093,FEF334C14BAE,4,0.5714,12,8,294,7,0.7417,0.9996,0.8511,11,105,36.7500,0.9163,1.0000,0.9895,0.0736,0.6168,0.3341
265094,FEF334C14BAE,4,0.9000,16,7,294,7,0.5327,0.9992,0.8228,17,105,42.0000,0.6180,1.0000,0.9453,0.1385,0.5574,0.3629
265095,FEF334C14BAE,4,0.5714,7,8,294,7,0.6707,0.9995,0.8455,11,105,36.7500,0.9838,1.0000,0.9979,0.0736,0.6168,0.3341
265096,FEF334C14BAE,4,0.7500,0,8,294,7,0.5589,0.9980,0.7538,11,105,36.7500,0.6943,1.0000,0.9424,0.0000,0.5574,0.2908


In [64]:
# TODO FIX 
dall.score.max()

1.0833333333333333

In [65]:
dall['score'] = dall.score.apply(lambda x: min(x, 1.))

In [66]:
fold = 0
dvalid = dall[dall.fold==fold]
dtrain = dall[dall.fold!=fold]

In [67]:
dtrain.keys()

Index(['id', 'fold', 'score', 'model', 'num_paras', 'num_words', 'num_gts',
       'sep_prob_min', 'sep_prob_max', 'sep_prob_mean', 'len_min', 'len_max',
       'len_mean', 'token_probs_min', 'token_probs_max', 'token_probs_mean',
       'bi_probs_min', 'bi_probs_max', 'bi_probs_mean'],
      dtype='object')

In [77]:
reg_cols =  [
              'num_words', 
              'num_paras',
              'sep_prob_min',
              'sep_prob_max',
              'sep_prob_mean',
              'len_min',
              'len_max',
              'len_mean', 
              'token_probs_min',
              'token_probs_max',
              'token_probs_mean',
              'bi_probs_min',
              'bi_probs_max',
              'bi_probs_mean',
        ]
cat_cols = [
            'model',
            ]
label_col = 'score'
cols = reg_cols + cat_cols
X_train = dtrain[cols]
y_train = dtrain[[label_col]]
X_valid = dvalid[cols]
y_valid = dvalid[[label_col]]

In [123]:
learning_rate = 0.1
num_boost_round = 2000
params = {
          # "objective": "binary",
          # "objective": "regression" if label_col is 'score' else 'binary',
          "objective": "cross_entropy",
          "metric": "auc",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": 10,
          "max_bin": 256,
          "feature_fraction": 0.9,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": True,
          "max_drop": 50,
          "min_child_samples": 200,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "bagging_freq": 5,
          "bagging_fraction": 0.9,
          "num_trees": 200,
          "subsample": 0.9
          }

In [124]:
d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
bst = lgb.train(params, d_train, num_boost_round, valid_sets=d_valid, 
                categorical_feature=cat_cols,
                verbose_eval=1,
                early_stopping_rounds=10)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.535716 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[1]	valid_0's auc: 0.773638
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.77781
[3]	valid_0's auc: 0.782886
[4]	valid_0's auc: 0.797524
[5]	valid_0's auc: 0.79264
[6]	valid_0's auc: 0.793958
[7]	valid_0's auc: 0.795454
[8]	valid_0's auc: 0.806046
[9]	valid_0's auc: 0.808276
[10]	valid_0's auc: 0.81111
[11]	valid_0's auc: 0.811396
[12]	valid_0's auc: 0.812071
[13]	valid_0's auc: 0.8

In [136]:
l = list(zip(bst.feature_name(), bst.feature_importance()))
l.sort(key=lambda x: -x[1])
l

[('model', 44),
 ('sep_prob_mean', 34),
 ('len_mean', 31),
 ('num_paras', 30),
 ('bi_probs_min', 27),
 ('num_words', 22),
 ('sep_prob_max', 21),
 ('token_probs_min', 19),
 ('len_min', 9),
 ('bi_probs_mean', 7),
 ('token_probs_mean', 4),
 ('bi_probs_max', 2),
 ('sep_prob_min', 1),
 ('len_max', 1),
 ('token_probs_max', 0)]

In [137]:
dvalid['pred'] = bst.predict(dvalid[cols])

In [138]:
dvalid = dvalid.sort_values(['id', 'pred'], ascending=[True, False])

In [139]:
dp = dvalid.groupby('id').first().reset_index()

In [140]:
dp['score'].mean()

0.7022290862133234

In [141]:
dvalid

,id,fold,score,model,num_paras,num_words,num_gts,sep_prob_min,sep_prob_max,sep_prob_mean,len_min,len_max,len_mean,token_probs_min,token_probs_max,token_probs_mean,bi_probs_min,bi_probs_max,bi_probs_mean,pred
0,0000D23A521A,0,0.5238,5,10,251,8,0.1773,0.9984,0.8401,13,65,25.1000,0.9856,1.0000,0.9976,0.0736,0.6168,0.3648,0.7073
12,0000D23A521A,0,0.3429,12,10,251,8,0.5666,0.9988,0.8730,6,65,25.1000,0.5612,1.0000,0.9319,0.0001,0.6168,0.2826,0.6976
10,0000D23A521A,0,0.5238,3,9,251,8,0.2271,0.9984,0.8453,13,65,27.8889,0.9998,1.0000,1.0000,0.0736,0.6168,0.3121,0.6939
11,0000D23A521A,0,0.5238,13,9,251,8,0.1254,0.9982,0.8226,13,65,27.8889,0.9995,1.0000,0.9999,0.0736,0.6168,0.3121,0.6932
7,0000D23A521A,0,0.4857,8,9,251,8,0.3092,0.9981,0.8506,13,65,27.8889,1.0000,1.0000,1.0000,0.0736,0.6168,0.3179,0.6902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8315,FFF1442D6698,0,0.5238,4,14,651,10,0.0002,0.9960,0.7836,1,182,46.5000,0.0580,1.0000,0.7189,0.0172,0.6168,0.2488,0.4897
8320,FFF1442D6698,0,0.6653,8,19,651,10,0.0001,0.9926,0.7051,1,136,34.2632,0.0533,1.0000,0.7279,0.0172,0.6168,0.2689,0.4834
8321,FFF1442D6698,0,0.4857,10,7,651,10,0.0288,0.9912,0.7116,4,303,93.0000,0.7139,1.0000,0.9591,0.0051,0.6168,0.2309,0.4354
8314,FFF1442D6698,0,0.3333,2,7,651,10,0.0002,0.9966,0.7409,4,355,93.0000,0.7210,1.0000,0.9601,0.0736,0.6168,0.2456,0.3783


In [142]:
dvalid[['id', 'score', 'model', 'pred']]

,id,score,model,pred
0,0000D23A521A,0.5238,5,0.7073
12,0000D23A521A,0.3429,12,0.6976
10,0000D23A521A,0.5238,3,0.6939
11,0000D23A521A,0.5238,13,0.6932
7,0000D23A521A,0.4857,8,0.6902
...,...,...,...,...
8315,FFF1442D6698,0.5238,4,0.4897
8320,FFF1442D6698,0.6653,8,0.4834
8321,FFF1442D6698,0.4857,10,0.4354
8314,FFF1442D6698,0.3333,2,0.3783


In [143]:
dvalid[dvalid.model==0][['id', 'model', 'score', 'pred']]

,id,model,score,pred
15,0000D23A521A,0,0.4857,0.6612
51559,001A03E06F3C,0,0.7333,0.8277
50505,004BE23B05BF,0,0.9333,0.6418
49451,0054850878E3,0,0.4800,0.6393
48397,00852F390697,0,0.4841,0.6911
...,...,...,...,...
12544,FF9114183593,0,0.8857,0.6626
11490,FF9DB9D43F72,0,0.6000,0.7118
10436,FFA381E58FC6,0,0.6250,0.6454
9382,FFC43F453EF6,0,0.4667,0.5465


In [144]:
dp[['id', 'model', 'score', 'pred']]

,id,model,score,pred
0,0000D23A521A,5,0.5238,0.7073
1,001A03E06F3C,2,0.7333,0.8395
2,004BE23B05BF,13,0.9500,0.7154
3,0054850878E3,7,0.3500,0.6621
4,00852F390697,14,0.5397,0.7352
...,...,...,...,...
3089,FF9114183593,15,0.7429,0.7180
3090,FF9DB9D43F72,13,0.5600,0.8124
3091,FFA381E58FC6,5,0.6667,0.6709
3092,FFC43F453EF6,8,0.3333,0.6783


In [145]:
fes = []
scores = []
for row1, row2 in zip(dvalid[dvalid.model==0].itertuples(), dp.itertuples()):
  if row2.pred - row1.pred > 0.2:
    scores.append(row2.score)
  else:
    scores.append(row1.score)

In [146]:
np.mean(scores)

0.7122306057294542